## Deliverable 2. Create a Customer Travel Destinations Map.

In [70]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [71]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Swan River,CA,52.1058,-101.2676,62.64,60,40,3.74,scattered clouds
1,1,Ushuaia,AR,-54.8000,-68.3000,37.06,65,0,5.75,clear sky
2,2,Busselton,AU,-33.6500,115.3333,49.08,79,100,8.30,overcast clouds
3,3,Kamenka,RU,51.3223,42.7678,66.42,75,0,8.32,clear sky
4,4,Praia Da Vitoria,PT,38.7333,-27.0667,68.25,83,40,6.38,scattered clouds


In [72]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

In [73]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &  
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Swan River,CA,52.1058,-101.2676,62.64,60,40,3.74,scattered clouds
3,3,Kamenka,RU,51.3223,42.7678,66.42,75,0,8.32,clear sky
4,4,Praia Da Vitoria,PT,38.7333,-27.0667,68.25,83,40,6.38,scattered clouds
5,5,Jiayuguan,CN,39.8167,98.3000,73.45,29,79,8.66,broken clouds
8,8,Arraial Do Cabo,BR,-22.9661,-42.0278,69.40,79,11,15.55,few clouds
11,11,Kapaa,US,22.0752,-159.3190,80.92,77,75,3.00,light rain
14,14,Victoria,HK,22.2855,114.1577,76.98,82,100,5.91,moderate rain
15,15,Albany,US,42.6001,-73.9662,82.35,91,100,6.33,light rain
16,16,Puerto Ayora,EC,-0.7393,-90.3518,73.54,92,100,5.99,overcast clouds
17,17,Atar,MR,20.5169,-13.0499,88.48,15,7,9.15,clear sky


In [74]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                480
City                   480
Country                478
Lat                    480
Lng                    480
Max Temp               480
Humidity               480
Cloudiness             480
Wind Speed             480
Current Description    480
dtype: int64

In [75]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_cities_df = preferred_cities_df.dropna(how='any')
clean_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Swan River,CA,52.1058,-101.2676,62.64,60,40,3.74,scattered clouds
3,3,Kamenka,RU,51.3223,42.7678,66.42,75,0,8.32,clear sky
4,4,Praia Da Vitoria,PT,38.7333,-27.0667,68.25,83,40,6.38,scattered clouds
5,5,Jiayuguan,CN,39.8167,98.3000,73.45,29,79,8.66,broken clouds
8,8,Arraial Do Cabo,BR,-22.9661,-42.0278,69.40,79,11,15.55,few clouds
...,...,...,...,...,...,...,...,...,...,...
675,675,Dhidhdhoo,MV,6.8833,73.1000,82.13,78,6,17.67,clear sky
676,676,Uruzgan,AF,32.8333,66.0000,66.58,18,0,7.92,clear sky
679,679,Vengurla,IN,15.8667,73.6333,81.55,86,100,2.39,light rain
680,680,Verkhnevilyuysk,RU,63.4453,120.3167,82.62,43,79,3.00,broken clouds


In [76]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Swan River,CA,62.64,scattered clouds,52.1058,-101.2676,
3,Kamenka,RU,66.42,clear sky,51.3223,42.7678,
4,Praia Da Vitoria,PT,68.25,scattered clouds,38.7333,-27.0667,
5,Jiayuguan,CN,73.45,broken clouds,39.8167,98.3000,
8,Arraial Do Cabo,BR,69.40,few clouds,-22.9661,-42.0278,
11,Kapaa,US,80.92,light rain,22.0752,-159.3190,
14,Victoria,HK,76.98,moderate rain,22.2855,114.1577,
15,Albany,US,82.35,light rain,42.6001,-73.9662,
16,Puerto Ayora,EC,73.54,overcast clouds,-0.7393,-90.3518,
17,Atar,MR,88.48,clear sky,20.5169,-13.0499,


In [77]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [78]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')
clean_hotel_df.count()

City                   478
Country                478
Max Temp               478
Current Description    478
Lat                    478
Lng                    478
Hotel Name             478
dtype: int64

In [79]:
# 8a. Create the output File (CSV)
output_data_file = "weatherpy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [80]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [86]:
# 11a. Add a marker layer for each city to the map. 
import gmaps
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))